## 特征提取和匹配

SIFT已经获取专利，所以OpenCV不用调用，我们用ORB来代替SIFT特征，在这篇文章里我们直接谈如何用代码实现。

### OpenCV的ORB特征

#### 第一步：导入库，图片，创建ORB对象

导入OpenCV库。

In [ ]:
import cv2 as cv
img1 = cv.imread('1.png')
img2 = cv.imread('2.png')
# 初始化ORB
orb = cv.ORB_create()

### 第二步：寻找关键点和描述子
创建之后，就是主要的寻找关键点和计算描述子：

In [ ]:
 # 寻找关键点
 kp1 = orb.detect(img1)
 kp2 = orb.detect(img2)

 # 计算描述符
 kp1, des1 = orb.compute(img1, kp1) # 计算哪张图片的用哪张图片的关键点。
 kp2, des2 = orb.compute(img2, kp2)

找到了关键点可以画出关键点看一看：

In [ ]:
# 画出关键点
outimg1 = cv.drawKeypoints(img1, keypoints=kp1, outImage=None)
outimg2 = cv.drawKeypoints(img2, keypoints=kp2, outImage=None)

# 这里是把两张图片在同一个窗口中显示。
import numpy as np
outimg3 = np.hstack([outimg1, outimg2])
cv.imshow("Key Points", outimg3)
cv.waitKey(0)

有了关键点和描述子，我们就要对这些点进行匹配。这里我们用的是Brute-Force匹配器，它获取第一组中一个特征的描述符，并通过一些距离计算与第二组中的所有其他特征匹配，最接近的一个被返回。
我们用Hamming距离对描述子进行匹配。

### 第三步：进行匹配

In [ ]:
 # 初始化 BFMatcher
 bf = cv.BFMatcher(cv.NORM_HAMMING)

 # 对描述子进行匹配
 matches = bf.match(des1, des2)

### 第四步：对匹配点进行筛选

In [ ]:
# 计算最大距离和最小距离
min_distance = matches[0].distance
max_distance = matches[0].distance
for x in matches:
    if x.distance < min_distance:
        min_distance = x.distance
    if x.distance > max_distance:
        max_distance = x.distance
        
'''
    当描述子之间的距离大于两倍的最小距离时，认为匹配有误。
    但有时候最小距离会非常小，所以设置一个经验值30作为下限。
'''
 good_match = []
 for x in matches:
     if x.distance <= max(2 * min_distance, 30):
         good_match.append(x)
        
draw_match(img1, img2, kp1, kp2, good_match)

### 第五步：绘制结果图
这里我们将绘制结果单独拿出来：

In [ ]:
def draw_match(img1, img2, kp1, kp2, match):
    outimage = cv.drawMatches(img1, kp1, img2, kp2, match, outImg=None)
    cv.imshow("Match Result", outimage)
    cv.waitKey(0)

In [ ]:
## 完整代码

In [3]:
import cv2 as cv

def ORB_Feature(img1, img2):

    # 初始化ORB
    orb = cv.ORB_create()

    # 寻找关键点
    kp1 = orb.detect(img1)
    kp2 = orb.detect(img2)

    # 计算描述符
    kp1, des1 = orb.compute(img1, kp1)
    kp2, des2 = orb.compute(img2, kp2)

    # 画出关键点
    outimg1 = cv.drawKeypoints(img1, keypoints=kp1, outImage=None)
    outimg2 = cv.drawKeypoints(img2, keypoints=kp2, outImage=None)
	
	# 显示关键点
    # import numpy as np
    # outimg3 = np.hstack([outimg1, outimg2])
    # cv.imshow("Key Points", outimg3)
    # cv.waitKey(0)

    # 初始化 BFMatcher
    bf = cv.BFMatcher(cv.NORM_HAMMING)

    # 对描述子进行匹配
    matches = bf.match(des1, des2)

    # 计算最大距离和最小距离
    min_distance = matches[0].distance
    max_distance = matches[0].distance
    for x in matches:
        if x.distance < min_distance:
            min_distance = x.distance
        if x.distance > max_distance:
            max_distance = x.distance

    # 筛选匹配点
    '''
        当描述子之间的距离大于两倍的最小距离时，认为匹配有误。
        但有时候最小距离会非常小，所以设置一个经验值30作为下限。
    '''
    good_match = []
    for x in matches:
        if x.distance <= max(2 * min_distance, 30):
            good_match.append(x)
            # print(x.distance)

    # 绘制匹配结果
    draw_match(img1, img2, kp1, kp2, good_match)

def draw_match(img1, img2, kp1, kp2, match):
    for i in range(0,len(match)):
        print(kp1[match[i].queryIdx].pt)
        print(kp2[match[i].queryIdx].pt)
        print("key point",i)
    # 显示图片
    # outimage = cv.drawMatches(img1, kp1, img2, kp2, match, outImg=None)
    # cv.imshow("Match Result", outimage)
    # cv.waitKey(0)

if __name__ == '__main__':
    # 读取图片
    image1 = cv.imread('1.png')
    image2 = cv.imread('2.png')
    ORB_Feature(image1, image2)
    print("finish")

(245.0, 211.0)
(115.0, 172.0)
key point 0
(46.0, 270.0)
(63.0, 293.0)
key point 1
(420.0, 169.0)
(50.0, 281.0)
key point 2
(143.0, 175.0)
(234.0, 117.0)
key point 3
(81.0, 312.0)
(444.0, 240.0)
key point 4
(527.0, 241.0)
(54.0, 285.0)
key point 5
(318.0, 92.0)
(369.0, 138.0)
key point 6
(76.0, 290.0)
(512.0, 266.0)
key point 7
(373.0, 217.0)
(510.0, 266.0)
key point 8
(295.0, 225.0)
(528.0, 272.0)
key point 9
(79.0, 307.0)
(224.0, 150.0)
key point 10
(540.0, 244.0)
(130.0, 210.0)
key point 11
(207.0, 288.0)
(281.0, 45.0)
key point 12
(49.0, 267.0)
(61.0, 287.0)
key point 13
(244.8000030517578, 210.00001525878906)
(444.0000305175781, 238.8000030517578)
key point 14
(428.4000244140625, 169.20001220703125)
(424.8000183105469, 159.60000610351562)
key point 15
(318.0, 93.60000610351562)
(334.8000183105469, 110.4000015258789)
key point 16
(331.20001220703125, 94.80000305175781)
(129.60000610351562, 207.60000610351562)
key point 17
(526.800048828125, 241.20001220703125)
(421.20001220703125, 1